## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [34]:
print (tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [35]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/drive/My Drive/R6/prices.csv", delimiter=",")

### Check all columns in the dataset

In [38]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],axis=1,inplace=True)

In [40]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data[:1000]

In [0]:
x = data.iloc[:,0:4]

In [43]:
y = data.iloc[:,4:]/1000000
y

,volume
0,2.1636
1,2.3864
2,2.4895
3,2.0063
4,1.4086
...,...
995,2.1332
996,1.9824
997,37.1528
998,6.5686


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20,random_state=3)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train =np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
Y_train =np.array(Y_train).astype('float32')
Y_test = np.array(Y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(X_train, W, b):
    y_pred = tf.add(tf.matmul(X_train, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(Y_train, y_predicted):
    diff = Y_train - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(X_train, Y_train, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(X_train, w, b)
        current_loss = loss(Y_train, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [51]:
for i in range(100):    
    W, b = train(X_train, Y_train, W, b)
    print('Current Training Loss on iteration', i, ' -- ', loss(Y_train, prediction(X_train, W, b)).numpy())

Current Training Loss on iteration 0  --  197.80934
Current Training Loss on iteration 1  --  195.78816
Current Training Loss on iteration 2  --  193.92549
Current Training Loss on iteration 3  --  192.2088
Current Training Loss on iteration 4  --  190.62675
Current Training Loss on iteration 5  --  189.1688
Current Training Loss on iteration 6  --  187.82506
Current Training Loss on iteration 7  --  186.58664
Current Training Loss on iteration 8  --  185.44536
Current Training Loss on iteration 9  --  184.39355
Current Training Loss on iteration 10  --  183.42416
Current Training Loss on iteration 11  --  182.5308
Current Training Loss on iteration 12  --  181.70747
Current Training Loss on iteration 13  --  180.94868
Current Training Loss on iteration 14  --  180.24937
Current Training Loss on iteration 15  --  179.60492
Current Training Loss on iteration 16  --  179.01096
Current Training Loss on iteration 17  --  178.4636
Current Training Loss on iteration 18  --  177.9591
Current 

In [52]:
for i in range(100):    
    W, b = train(X_train, Y_train, W, b)
    if i%5 == 0:
      print('Current Training Loss on iteration : ', i,' -- ', loss(Y_train, prediction(X_train, W, b)).numpy())
    

Current Training Loss on iteration :  0  --  172.03566
Current Training Loss on iteration :  5  --  172.03316
Current Training Loss on iteration :  10  --  172.03152
Current Training Loss on iteration :  15  --  172.03041
Current Training Loss on iteration :  20  --  172.02965
Current Training Loss on iteration :  25  --  172.0291
Current Training Loss on iteration :  30  --  172.02872
Current Training Loss on iteration :  35  --  172.02852
Current Training Loss on iteration :  40  --  172.02837
Current Training Loss on iteration :  45  --  172.0282
Current Training Loss on iteration :  50  --  172.0281
Current Training Loss on iteration :  55  --  172.02803
Current Training Loss on iteration :  60  --  172.02795
Current Training Loss on iteration :  65  --  172.0279
Current Training Loss on iteration :  70  --  172.0279
Current Training Loss on iteration :  75  --  172.02782
Current Training Loss on iteration :  80  --  172.02776
Current Training Loss on iteration :  85  --  172.0277


### Get the shapes and values of W and b

In [53]:
print ('shape of W : ', W)

shape of W :  tf.Tensor(
[[1.2862536]
 [1.3554295]
 [1.2863494]
 [1.3583087]], shape=(4, 1), dtype=float32)


In [54]:
print ('shape of W : ', b)

shape of W :  tf.Tensor([2.6439803], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [55]:
pred=prediction(X_test,W,b)
pred

<tf.Tensor: id=3296, shape=(200, 1), dtype=float32, numpy=
array([[5.287775 ],
       [5.2875805],
       [5.2874413],
       [5.2872868],
       [5.2873816],
       [5.288076 ],
       [5.287963 ],
       [5.2875824],
       [5.287302 ],
       [5.2872744],
       [5.287551 ],
       [5.2878036],
       [5.287967 ],
       [5.2871723],
       [5.2877884],
       [5.287678 ],
       [5.287855 ],
       [5.28751  ],
       [5.287779 ],
       [5.287405 ],
       [5.287567 ],
       [5.2871876],
       [5.2871556],
       [5.2872806],
       [5.2874117],
       [5.287141 ],
       [5.287696 ],
       [5.28675  ],
       [5.286813 ],
       [5.2871933],
       [5.2879047],
       [5.287507 ],
       [5.2879295],
       [5.2877893],
       [5.287151 ],
       [5.2876463],
       [5.2872586],
       [5.2872095],
       [5.287731 ],
       [5.2872295],
       [5.287771 ],
       [5.2864695],
       [5.287985 ],
       [5.287569 ],
       [5.28723  ],
       [5.2872   ],
       [5.287592 ],
 

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("/content/drive/My Drive/R6/11_Iris.csv", delimiter=",")

In [75]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [0]:
iris.shape

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
iris.Species = le.fit_transform(iris.Species)

In [0]:
y_iris = tf.keras.utils.to_categorical(iris.Species, num_classes=3)

### Splitting the data into feature set and target set

In [67]:
X_iris = iris.iloc[:,1:5]
#y_iris = iris.Species
y_iris.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:

# Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

# Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X_iris, y_iris, test_size=0.20,random_state=3)

### Model Training 

In [71]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=30)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 3ms/sample - loss: 1.0963 - accuracy: 0.4583 - val_loss: 1.0462 - val_accuracy: 0.6667
Epoch 2/30
120/120 [==============================] - 0s 92us/sample - loss: 1.0600 - accuracy: 0.4750 - val_loss: 1.0573 - val_accuracy: 0.6667
Epoch 3/30
120/120 [==============================] - 0s 90us/sample - loss: 1.0069 - accuracy: 0.5583 - val_loss: 0.9950 - val_accuracy: 0.6667
Epoch 4/30
120/120 [==============================] - 0s 82us/sample - loss: 0.9677 - accuracy: 0.5333 - val_loss: 0.9416 - val_accuracy: 0.6667
Epoch 5/30
120/120 [==============================] - 0s 86us/sample - loss: 0.9285 - accuracy: 0.5667 - val_loss: 0.9045 - val_accuracy: 0.6667
Epoch 6/30
120/120 [==============================] - 0s 85us/sample - loss: 0.8924 - accuracy: 0.5833 - val_loss: 0.8779 - val_accuracy: 0.6667
Epoch 7/30
120/120 [==============================] - 0s 83us/sample - loss: 0.8645 - 

### Model Prediction

In [72]:
model.predict(testX)

array([[0.9071611 , 0.04966755, 0.04317131],
       [0.88368726, 0.06179665, 0.0545161 ],
       [0.9261255 , 0.03942805, 0.03444648],
       [0.8739863 , 0.06654799, 0.05946573],
       [0.97518593, 0.01354037, 0.01127364],
       [0.01038915, 0.3923731 , 0.59723777],
       [0.06469466, 0.42161405, 0.51369125],
       [0.91837287, 0.04377497, 0.03785216],
       [0.02083757, 0.41173762, 0.5674248 ],
       [0.0963858 , 0.40491384, 0.49870044],
       [0.0937999 , 0.4115612 , 0.49463892],
       [0.89511377, 0.05610836, 0.04877783],
       [0.08438455, 0.4150718 , 0.5005437 ],
       [0.05332894, 0.42455208, 0.522119  ],
       [0.01828641, 0.40832946, 0.57338417],
       [0.94440806, 0.03004513, 0.02554679],
       [0.02737702, 0.41835982, 0.5542632 ],
       [0.01360943, 0.4001931 , 0.5861975 ],
       [0.02704453, 0.4180803 , 0.5548752 ],
       [0.9128187 , 0.04683953, 0.04034187],
       [0.02762198, 0.41856247, 0.5538156 ],
       [0.01936698, 0.4098442 , 0.57078886],
       [0.

### Save the Model

In [0]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?